In [1]:
import re
from functools import reduce
import itertools
from collections import Counter
import numpy as np
from dataclasses import dataclass


@dataclass
class Node():
    name:str
    left:str
    right:str



def get_input(n):
    with open('input_'+n+'.txt', 'r') as infile:
        return infile.read().strip()

def parse_input(puzzle):
    instructions,body = puzzle.split('\n\n')

    def get_nodes(lines):
        nodes = {}
        for l in lines:
            name, left,right = re.search(r'(\w+)\s*=\s*\((\w+),\s*(\w+)\)',l).groups()
            nodes[name] = Node(name,left,right)
        return nodes
    return instructions.strip(), get_nodes(body.split('\n'))

def solve1(puzzle):
    instructions, nodes = parse_input(puzzle)

    def traverse(instructions, nodes):
        current_node = nodes['AAA']
        steps = 0
        for step in itertools.cycle(instructions):
            if current_node.name == 'ZZZ':
                break
            next_name = current_node.left if step == 'L' else current_node.right
            current_node = nodes[next_name]
            steps +=1
        return steps
        
    print(traverse(instructions,nodes))

def solve2(puzzle):
    pass
    

In [2]:
a = [1,23,4]
b = a[2]
b= 2 
a

[1, 23, 4]

In [3]:
sample1 = """RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)"""

sample2 = """LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)"""

solve1(sample1)
solve1(sample2)

2
6


In [6]:
%%time
puzzle = get_input('8')
#print(puzzle)
solve1(puzzle)

18727
CPU times: user 7.19 ms, sys: 0 ns, total: 7.19 ms
Wall time: 7.28 ms


In [58]:
sample3 = """LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)"""



sample4 = """L

11A = (11B, XXX)
11B = (11C, XXX)
11C = (11Z, XXX)
11Z = (11A, XXX)
22A = (22B, XXX)
22B = (22C, XXX)
22C = (22D, XXX)
22D = (22Z, XXX)
22Z = (22A, XXX)"""

sample5 = """L

11A = (11B, XXX)
11B = (11C, XXX)
11C = (11Z, XXX)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, XXX)
22C = (22D, XXX)
22D = (22Z, XXX)
22Z = (22A, XXX)"""

In [94]:
from pprint import pprint

def solve2(puzzle):
    instructions, nodes = parse_input(puzzle)


    def get_loop_length(node):

        def traverse():
            current_node = node
            for steps, step in enumerate(itertools.cycle(instructions)):
                yield steps, current_node
                next_name = current_node.left if step == 'L' else current_node.right
                current_node = nodes[next_name]
                
        states = set()
        end_positions = []
        last_node = Node('','','')
        for steps, current_node in traverse():    
            state = (current_node.name, steps%len(instructions))
            if state in states:
                print(steps, last_node.name)
                break
            states.add(state)
            last_node = current_node
            if current_node.name.endswith('Z'):
                print(steps, current_node)
                end_positions.append(steps)

        l = steps
        state_loop = state

        # get x0

        for steps, node in traverse():
            state = (node.name, steps%len(instructions))
            if state == state_loop:
                break
            

        x0 = steps

        l_loop = l - x0
        #print(f'Found loop for node {node.name} after {steps} steps')
        return x0, l_loop, end_positions

    lengths = [get_loop_length(n) for n in nodes.values() if n.name.endswith('A')]
    pprint(lengths)

    def is_endposition(nsteps, loop_length, end_positions):
        return (loop_length-len(instructions))
    
    l_max = max(lengths, key = lambda x: x[1])[1]

    lcm = reduce(lambda a,b: a*b, ( li[1] for li in lengths ) )
    return lcm
            
solve2(sample5)  

3 Node(name='11Z', left='11B', right='XXX')
4 11Z
4 Node(name='22Z', left='22A', right='XXX')
5 22Z
[(1, 3, [3]), (0, 5, [4])]


15

In [95]:
print(solve2(puzzle))

22411 Node(name='TKZ', left='BHR', right='BGC')
22415 BBG
18727 Node(name='ZZZ', left='FBF', right='GFB')
18731 DPD
24253 Node(name='LLZ', left='HCS', right='VHR')
24255 HCS
14429 Node(name='JLZ', left='KMM', right='QCG')
14431 KMM
16271 Node(name='KJZ', left='VQH', right='CTG')
16276 TDP
20569 Node(name='HVZ', left='QRH', right='VHK')
20574 BJN
[(4, 22411, [22411]),
 (4, 18727, [18727]),
 (2, 24253, [24253]),
 (2, 14429, [14429]),
 (5, 16271, [16271]),
 (5, 20569, [20569])]
49153966541198323241619811


In [ ]:
l0 = 